In [11]:
import sys
import cv2
from cvzone.PoseModule import PoseDetector
import numpy as np

imgPath = './images/template_01/'
img_A = cv2.imread(imgPath + 'A.png')
img_B = cv2.imread(imgPath + 'B.png')
img_out = cv2.imread(imgPath + 'A.png')

height = img_A.shape[0]
width = img_A.shape[1]
print(height,width)
if img_B.shape[0] != height or img_B.shape[1] != width:
    print("Error: two images have different sizes")
    sys.exit()   

# temptlate (offset, size) (y,x)
tplt = np.array([[600,   420,  472, 2530],  # block0 : Head
                 [1110,  420, 2340, 4100],  # block1 : Pic
                 [3480,  420,  820, 4520],  # block2 : Paragraph
                 [140,  5300,  800, 2500],  # block3 : Weather
                 [1240, 5300, 1250, 2490],  # block4 : Edit fav
                 [2890, 5300, 1500, 2490]]) # block5 : Comment


# resizing
reduceFactor = 4
img_A = cv2.resize(img_A, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
img_B = cv2.resize(img_B, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
img_out = cv2.resize(img_out, None, fx=(1/reduceFactor), fy=(1/reduceFactor))
tplt = tplt//reduceFactor



img_segment_0A = img_A[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :]
img_segment_0B = img_B[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :]
img_segment_1A = img_A[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :]
img_segment_1B = img_B[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :]
img_segment_2A = img_A[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :]
img_segment_2B = img_B[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :]
img_segment_3A = img_A[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :]
img_segment_3B = img_B[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :]
img_segment_4A = img_A[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :]
img_segment_4B = img_B[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :]
img_segment_5A = img_A[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :]
img_segment_5B = img_B[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :]




duration = 60
blockState = [1,5,10,15,20,30]
blockTrigger = [1,1,1,1,1,1]


# 0 -> A (still)
# 1~99 -> changing from A to B
# 100 -> B (still)
# 101~199 -> changing from B to A

# 200 (0) -> A (still) (return to 0: 200 % 200 = 0)



while True:
    
    # display according to blockState[]
    
# block 0
    #ct = blockState[0]
    if blockState[0] == 0:
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = img_segment_0A
        if blockTrigger[0] == 1:
            blockState[0] += 1
            
    elif blockState[0] == duration:
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = img_segment_0B
        if blockTrigger[0] == 1:
            blockState[0] += 1
            
    elif blockState[0] < duration: # render A->B
        #print("A->B")
        alpha = blockState[0] / duration
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = cv2.addWeighted(img_segment_0B, alpha, img_segment_0A, (1-alpha), 0)
        blockState[0] += 1
        if blockState[0] == duration:
            blockTrigger = 0
            
            
            ###
            
            
            print("finish A->B (block 0)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[0]-duration) / duration
        img_out[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :] = cv2.addWeighted(img_segment_0A, alpha, img_segment_0B, (1-alpha), 0)
        blockState[0] += 1
        if blockState[0] == (duration*2):
            print("finish B->A (block 0)")
            #break 
    
    blockState[0] = blockState[0] % (duration * 2)
###
    
    
# block1   
    if blockState[1] == 0:
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = img_segment_1A
        if blockTrigger[1] == 1:
            blockState[1] += 1
            
    elif blockState[1] == duration:
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = img_segment_1B
        if blockTrigger[1] == 1:
            blockState[1] += 1
            
    elif blockState[1] < duration: # render A->B
        #print("A->B")
        alpha = blockState[1] / duration
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = cv2.addWeighted(img_segment_1B, alpha, img_segment_1A, (1-alpha), 0)
        blockState[1] += 1
        if blockState[1] == duration:
            print("finish A->B (block 1)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[1]-duration) / duration
        img_out[tplt[1,0]:tplt[1,0]+tplt[1,2], tplt[1,1]:tplt[1,1]+tplt[1,3], :] = cv2.addWeighted(img_segment_1A, alpha, img_segment_1B, (1-alpha), 0)
        blockState[1] += 1
        if blockState[1] == (duration*2):
            print("finish B->A (block 1)")
            #break 
    
    blockState[1] = blockState[1] % (duration * 2) 
###  
    
# block2
    if blockState[2] == 0:
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = img_segment_2A
        if blockTrigger[2] == 1:
            blockState[2] += 1
            
    elif blockState[2] == duration:
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = img_segment_2B
        if blockTrigger[2] == 1:
            blockState[2] += 1
            
    elif blockState[2] < duration: # render A->B
        #print("A->B")
        alpha = blockState[2] / duration
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = cv2.addWeighted(img_segment_2B, alpha, img_segment_2A, (1-alpha), 0)
        blockState[2] += 1
        if blockState[2] == duration:
            print("finish A->B (block 2)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[2]-duration) / duration
        img_out[tplt[2,0]:tplt[2,0]+tplt[2,2], tplt[2,1]:tplt[2,1]+tplt[2,3], :] = cv2.addWeighted(img_segment_2A, alpha, img_segment_2B, (1-alpha), 0)
        blockState[2] += 1
        if blockState[2] == (duration*2):
            print("finish B->A (block 2)")
            #break 
    
    blockState[2] = blockState[2] % (duration * 2) 
###



# block3
    if blockState[3] == 0:
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = img_segment_3A
        if blockTrigger[3] == 1:
            blockState[3] += 1
            
    elif blockState[3] == duration:
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = img_segment_3B
        if blockTrigger[3] == 1:
            blockState[3] += 1
            
    elif blockState[3] < duration: # render A->B
        #print("A->B")
        alpha = blockState[3] / duration
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = cv2.addWeighted(img_segment_3B, alpha, img_segment_3A, (1-alpha), 0)
        blockState[3] += 1
        if blockState[3] == duration:
            print("finish A->B (block 3)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[3]-duration) / duration
        img_out[tplt[3,0]:tplt[3,0]+tplt[3,2], tplt[3,1]:tplt[3,1]+tplt[3,3], :] = cv2.addWeighted(img_segment_3A, alpha, img_segment_3B, (1-alpha), 0)
        blockState[3] += 1
        if blockState[3] == (duration*2):
            print("finish B->A (block 3)")
            #break 
    
    blockState[3] = blockState[3] % (duration * 2) 
###




# block4
    if blockState[4] == 0:
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = img_segment_4A
        if blockTrigger[4] == 1:
            blockState[4] += 1
            
    elif blockState[4] == duration:
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = img_segment_4B
        if blockTrigger[4] == 1:
            blockState[4] += 1
            
    elif blockState[4] < duration: # render A->B
        #print("A->B")
        alpha = blockState[4] / duration
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = cv2.addWeighted(img_segment_4B, alpha, img_segment_4A, (1-alpha), 0)
        blockState[4] += 1
        if blockState[4] == duration:
            print("finish A->B (block 4)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[4]-duration) / duration
        img_out[tplt[4,0]:tplt[4,0]+tplt[4,2], tplt[4,1]:tplt[4,1]+tplt[4,3], :] = cv2.addWeighted(img_segment_4A, alpha, img_segment_4B, (1-alpha), 0)
        blockState[4] += 1
        if blockState[4] == (duration*2):
            print("finish B->A (block 4)")
            #break 
    
    blockState[4] = blockState[4] % (duration * 2) 
###
    
    
    
# block5
    if blockState[5] == 0:
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = img_segment_5A
        if blockTrigger[5] == 1:
            blockState[5] += 1
            
    elif blockState[5] == duration:
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = img_segment_5B
        if blockTrigger[5] == 1:
            blockState[5] += 1
            
    elif blockState[5] < duration: # render A->B
        #print("A->B")
        alpha = blockState[5] / duration
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = cv2.addWeighted(img_segment_5B, alpha, img_segment_5A, (1-alpha), 0)
        blockState[5] += 1
        if blockState[5] == duration:
            print("finish A->B (block 5)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[5]-duration) / duration
        img_out[tplt[5,0]:tplt[5,0]+tplt[5,2], tplt[5,1]:tplt[5,1]+tplt[5,3], :] = cv2.addWeighted(img_segment_5A, alpha, img_segment_5B, (1-alpha), 0)
        blockState[5] += 1
        if blockState[5] == (duration*2):
            print("finish B->A (block 5)")
            #break 
    
    blockState[5] = blockState[5] % (duration * 2) 
###




    # update blockTrigger[] depending on the cvzone result
    
    


    
    
    #cv2.imshow('img',cv2.resize(img_out, None, fx=reduceFactor, fy=reduceFactor))
    cv2.imshow('img',img_out)
    cv2.waitKey(1) 
    
    #'''
    key = cv2.waitKey(1) # 30??
    if key & 0xFF == ord('q'): # 27=ESC 113='q'
        print("break", key)
        break
    #'''
        
### end of main loop

    
    
    
    

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1) 









4500 8000
break 113


-1

In [8]:
import logging
import sys
print(111)
#logging.exception("two images have different sizes")
#sys.exit()
print(22)

111
22


In [12]:
# temptlate
tplt = np.array([[0,   0,   100, 100], # block0 (offset, size) (y,x)
        [100, 100, 100, 100], # block1
        [200, 200, 100, 100], # block2
        [300, 300, 100, 100], # block3
        [400, 400, 100, 100], # block4
        [500, 500, 100, 100]]) # block5

print(tplt[0,0])

0


In [7]:
img_segment_1A = img_A[tplt[0,0]:tplt[0,0]+tplt[0,2], tplt[0,1]:tplt[0,1]+tplt[0,3], :]

cv2.imshow('img',img_segment_1A)



cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1) 


-1

In [2]:
import cv2
key = 88
while True:
    
#    print(key)
    
    key = cv2.waitKey(30) # 30??
    if key == ord('q') or key == 27: # 27=ESC 113='q'
        print("break", key)
        key = 999
        break

In [11]:


def add10(x,y):
    return 10*(x+y)
add10(3,2)

50

In [21]:
tplt//4

array([[ 150,  105,  118,  632],
       [ 277,  105,  585, 1025],
       [ 870,  105,  205, 1130],
       [  35, 1325,  200,  625],
       [ 310, 1325,  312,  622],
       [ 722, 1325,  375,  622]])

In [ ]:
# block$
    if blockState[$] == 0:
        img_out[tplt[$,0]:tplt[$,0]+tplt[$,2], tplt[$,1]:tplt[$,1]+tplt[$,3], :] = img_segment_$A
        if blockTrigger[$] == 1:
            blockState[$] += 1
            
    elif blockState[$] == duration:
        img_out[tplt[$,0]:tplt[$,0]+tplt[$,2], tplt[$,1]:tplt[$,1]+tplt[$,3], :] = img_segment_$B
        if blockTrigger[$] == 1:
            blockState[$] += 1
            
    elif blockState[$] < duration: # render A->B
        #print("A->B")
        alpha = blockState[$] / duration
        img_out[tplt[$,0]:tplt[$,0]+tplt[$,2], tplt[$,1]:tplt[$,1]+tplt[$,3], :] = cv2.addWeighted(img_segment_$B, alpha, img_segment_$A, (1-alpha), 0)
        blockState[$] += 1
        if blockState[$] == duration:
            print("finish A->B (block $)")
            #break
        
    else: # render B->A
        #print("B->A")
        alpha = (blockState[$]-duration) / duration
        img_out[tplt[$,0]:tplt[$,0]+tplt[$,2], tplt[$,1]:tplt[$,1]+tplt[$,3], :] = cv2.addWeighted(img_segment_$A, alpha, img_segment_$B, (1-alpha), 0)
        blockState[$] += 1
        if blockState[$] == (duration*2):
            print("finish B->A (block $)")
            #break 
    
    blockState[$] = blockState[$] % (duration * 2) 
###